## Importación de librerías

Se importan las bibliotecas necesarias para el proyecto: TensorFlow/Keras para el modelo de red neuronal, NumPy y pandas para manejo de datos, collections.Counter para conteo de letras, requests y json para descargar datasets de palabras en español, y re para expresiones regulares.

In [1]:
"""
Proyecto Final - Inteligencia Artificial
Asistente para Wordle en Español (La palabra del día)
"""

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
from collections import Counter
import requests
import json
import re

2025-12-23 11:46:04.513817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766490364.819342      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766490364.916498      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

## Descarga de palabras

Define una única función para descargar, ya que probando a descargar desde fuentes externas como Github tenía errores, además de que no he encontrado una API oficial de la RAE ni la API de donde la propia página de 'La palabra del día' saca sus palabras. También incluye una función para filtrar palabras válidas para Wordle: exactamente 5 letras, sin tildes, sin caracteres especiales y con la letra Ñ.

In [2]:
def descargar_palabras_api():
    """
    Descarga palabras de la API de palabras aleatorias españolas
    """
    # Esta es una fuente más limitada pero funcional
    url = "https://raw.githubusercontent.com/words/an-array-of-spanish-words/master/index.json"
    try:
        response = requests.get(url)
        response.raise_for_status()
        palabras = json.loads(response.text)
        print(f"Descargadas {len(palabras)} palabras de API")
        return palabras
    except:
        raise Exception("No se pudieron descargar las palabras de ninguna fuente")

def filtrar_palabras_wordle(palabras):
    """
    Filtra palabras para Wordle: 5 letras, sin tildes, sin caracteres especiales
    """
    palabras_validas = []
    
    for palabra in palabras:
        palabra = palabra.strip().upper()
        
        # Debe tener exactamente 5 letras
        if len(palabra) != 5:
            continue
        
        # Solo letras (sin números ni caracteres especiales)
        if not palabra.isalpha():
            continue
        
        # Sin tildes
        if any(c in palabra for c in 'ÁÉÍÓÚÜ'):
            continue
        
        palabras_validas.append(palabra)
    
    # Eliminar duplicados
    palabras_validas = list(set(palabras_validas))
    
    print(f"Palabras válidas para Wordle: {len(palabras_validas)}")
    return sorted(palabras_validas)

## Procesamiento y análisis de letras

Contiene funciones para analizar la frecuencia de cada letra por posición en las palabras válidas y para crear un vector de características por palabra que incluye frecuencia de letras, score por posición, diversidad de letras y score de frecuencia global.

In [3]:
def crear_estadisticas_letras(palabras):
    """
    Analiza frecuencia de letras por posición
    """
    estadisticas = {i: Counter() for i in range(5)}
    frecuencia_global = Counter()
    
    for palabra in palabras:
        for i, letra in enumerate(palabra):
            estadisticas[i][letra] += 1
            frecuencia_global[letra] += 1
    
    return estadisticas, frecuencia_global

def crear_features_palabra(palabra, estadisticas, frecuencia_global):
    """
    Crea vector de características para una palabra
    Features:
    - 26 valores: frecuencia de cada letra del alfabeto
    - 5 valores: score de cada posición
    - 1 valor: diversidad de letras
    - 1 valor: score total de frecuencia
    """
    alfabeto = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    features = []
    
    # Frecuencia de cada letra en la palabra
    for letra in alfabeto:
        features.append(palabra.count(letra))
    
    # Score de posición para cada letra
    for i, letra in enumerate(palabra):
        score_pos = estadisticas[i].get(letra, 0) / len(estadisticas[i])
        features.append(score_pos)
    
    # Diversidad de letras (penaliza letras repetidas)
    diversidad = len(set(palabra)) / 5.0
    features.append(diversidad)
    
    # Score de frecuencia global
    score_freq = sum(frecuencia_global.get(letra, 0) for letra in palabra)
    score_freq_norm = score_freq / sum(frecuencia_global.values())
    features.append(score_freq_norm)
    
    return np.array(features, dtype=np.float32)

## Creación del modelo de red neuronal

Define la arquitectura de la red neuronal para clasificar palabras según su probabilidad de ser la respuesta correcta, utilizando capas densas y dropout para regularización.

In [4]:
def crear_modelo():
    """
    Red neuronal para scoring de palabras
    """
    modelo = keras.Sequential([
        layers.Input(shape=(33,)),  # 26 + 5 + 1 + 1
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Score de 0 a 1
    ])
    
    modelo.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return modelo

def generar_datos_entrenamiento(palabras, estadisticas, frecuencia_global):
    """
    Genera datos de entrenamiento basados en características lingüísticas
    """
    X = []
    y = []
    
    # Calculamos scores para todas las palabras
    scores = []
    for palabra in palabras:
        # Score basado en frecuencia y diversidad
        score_freq = sum(frecuencia_global.get(letra, 0) for letra in palabra)
        diversidad = len(set(palabra)) / 5.0
        score = (score_freq / sum(frecuencia_global.values())) * diversidad
        scores.append(score)
    
    # Normalizamos scores
    max_score = max(scores)
    min_score = min(scores)
    
    for palabra, score in zip(palabras, scores):
        features = crear_features_palabra(palabra, estadisticas, frecuencia_global)
        X.append(features)
        # Normalizar score a [0, 1]
        y.append((score - min_score) / (max_score - min_score))
    
    return np.array(X), np.array(y)

## Motor de sugerencias

Implementa la clase principal del asistente que gestiona el estado del juego, filtra palabras candidatas según el feedback recibido y genera sugerencias basadas en probabilidad o en maximizar información.

In [5]:
class WordleAssistant:
    def __init__(self, palabras, modelo, estadisticas, frecuencia_global):
        self.palabras = palabras
        self.modelo = modelo
        self.estadisticas = estadisticas
        self.frecuencia_global = frecuencia_global
        self.letras_verdes = {}  # {posicion: letra}
        self.letras_amarillas = set()  # letras que están pero en otra posición
        self.letras_amarillas_pos = {}  # {letra: [posiciones_donde_NO_está]}
        self.letras_negras = set()
    
    def actualizar_feedback(self, palabra, feedback):
        """
        Actualiza el estado con el feedback del intento
        palabra: str, la palabra intentada
        feedback: str, 5 caracteres con 'V'(verde), 'A'(amarillo), 'N'(negro)
        Ejemplo: "VNANV" significa verde-negro-amarillo-negro-verde
        """
        palabra = palabra.upper()
        feedback = feedback.upper()
        
        for i, (letra, color) in enumerate(zip(palabra, feedback)):
            if color == 'V':
                self.letras_verdes[i] = letra
                self.letras_amarillas.discard(letra)  # Ya no es amarilla
            elif color == 'A':
                self.letras_amarillas.add(letra)
                if letra not in self.letras_amarillas_pos:
                    self.letras_amarillas_pos[letra] = []
                self.letras_amarillas_pos[letra].append(i)
            elif color == 'N':
                # Solo marcar como negra si no está verde o amarilla
                if letra not in self.letras_verdes.values() and letra not in self.letras_amarillas:
                    self.letras_negras.add(letra)
    
    def filtrar_candidatas(self):
        """
        Filtra palabras según restricciones actuales
        """
        candidatas = []
        
        for palabra in self.palabras:
            valida = True
            
            # Verificar letras verdes (deben estar en su posición)
            for pos, letra in self.letras_verdes.items():
                if palabra[pos] != letra:
                    valida = False
                    break
            
            if not valida:
                continue
            
            # Verificar letras amarillas (deben estar pero no en ciertas posiciones)
            for letra in self.letras_amarillas:
                if letra not in palabra:
                    valida = False
                    break
                # Verificar que no esté en posiciones descartadas
                if letra in self.letras_amarillas_pos:
                    for pos_no in self.letras_amarillas_pos[letra]:
                        if palabra[pos_no] == letra:
                            valida = False
                            break
            
            if not valida:
                continue
            
            # Verificar letras negras (no deben estar)
            for letra in self.letras_negras:
                if letra in palabra:
                    valida = False
                    break
            
            if valida:
                candidatas.append(palabra)
        
        return candidatas
    
    def sugerir_palabra_probable(self, top_n=5):
        """
        Sugiere las palabras más probables de ser la respuesta
        """
        candidatas = self.filtrar_candidatas()
        
        if not candidatas:
            return []
        
        # Crear features y predecir
        features = np.array([
            crear_features_palabra(p, self.estadisticas, self.frecuencia_global)
            for p in candidatas
        ])
        
        scores = self.modelo.predict(features, verbose=0).flatten()
        
        # Ordenar por score
        indices_ordenados = np.argsort(scores)[::-1]
        
        return [(candidatas[i], float(scores[i])) for i in indices_ordenados[:top_n]]
    
    def sugerir_palabra_exploradora(self, top_n=5):
        """
        Sugiere palabras que maximicen la información (más letras diferentes)
        """
        candidatas = self.filtrar_candidatas()
        
        if not candidatas:
            return []
        
        # Calcular score de exploración
        scores_exploracion = []
        for palabra in candidatas:
            # Contar letras únicas no descubiertas
            letras_conocidas = set(self.letras_verdes.values()) | self.letras_amarillas | self.letras_negras
            letras_nuevas = set(palabra) - letras_conocidas
            # Score: cantidad de letras nuevas * frecuencia promedio
            score = len(letras_nuevas) * sum(self.frecuencia_global.get(l, 0) for l in letras_nuevas)
            scores_exploracion.append(score)
        
        # Ordenar por score de exploración
        indices_ordenados = np.argsort(scores_exploracion)[::-1]
        
        return [(candidatas[i], float(scores_exploracion[i])) for i in indices_ordenados[:top_n]]
    
    def reset(self):
        """Reinicia el asistente para una nueva partida"""
        self.letras_verdes = {}
        self.letras_amarillas = set()
        self.letras_amarillas_pos = {}
        self.letras_negras = set()

## Interfaz

Proporciona funciones para procesar entradas del usuario en formato PALABRA/COLORES y un modo interactivo que permite jugar múltiples partidas con el asistente.

In [6]:
def procesar_intento(asistente, entrada):
    """
    Procesa un intento en formato: PALABRA/COLORES
    """
    try:
        palabra, colores = entrada.upper().split('/')
        
        if len(palabra) != 5:
            return "Error: La palabra debe tener 5 letras"
        
        if len(colores) != 5:
            return "Error: Debes proporcionar 5 colores (V/A/N)"
        
        if not all(c in 'VAN' for c in colores):
            return "Error: Los colores deben ser V (verde), A (amarillo) o N (negro)"
        
        # Actualizar feedback
        asistente.actualizar_feedback(palabra, colores)
        
        # Obtener sugerencias
        candidatas_total = len(asistente.filtrar_candidatas())
        
        resultado = f"\n{'='*70}\n"
        resultado += f"Intento procesado: {palabra} → {colores}\n"
        resultado += f"{'='*70}\n"
        
        resultado += f"\nCandidatas restantes: {candidatas_total}\n"
        
        if candidatas_total == 0:
            resultado += "\nNo quedan candidatas válidas. Verifica los colores ingresados.\n"
            return resultado
        
        # Palabras más probables
        resultado += "\nTOP 5 PALABRAS MÁS PROBABLES:\n"
        probables = asistente.sugerir_palabra_probable(top_n=5)
        for i, (pal, score) in enumerate(probables, 1):
            resultado += f"   {i}. {pal} (confianza: {score:.4f})\n"
        
        # Palabras exploradoras
        resultado += "\nTOP 5 PALABRAS QUE APORTAN MÁS INFORMACIÓN:\n"
        exploradoras = asistente.sugerir_palabra_exploradora(top_n=5)
        for i, (pal, score) in enumerate(exploradoras, 1):
            resultado += f"   {i}. {pal} (info score: {score:.1f})\n"
        
        # Estado actual
        resultado += f"\nESTADO ACTUAL:\n"
        if asistente.letras_verdes:
            resultado += f"   Verdes: {dict(asistente.letras_verdes)}\n"
        if asistente.letras_amarillas:
            resultado += f"   Amarillas: {asistente.letras_amarillas}\n"
        if asistente.letras_negras:
            resultado += f"   Negras: {asistente.letras_negras}\n"
        
        return resultado
        
    except ValueError:
        return "Error: Formato incorrecto. Usa: PALABRA/COLORES (ej: CANTO/VAANN)"

def modo_interactivo(asistente):
    """
    Modo interactivo para jugar con el asistente
    """
    print("\n" + "="*70)
    print("MODO INTERACTIVO - ASISTENTE 'LA PALABRA DEL DÍA'")
    print("="*70)
    print("\nINSTRUCCIONES:")
    print("   - Introduce tu palabra y el resultado de colores")
    print("   - Formato: PALABRA/COLORES")
    print("   - Ejemplo: CANTO/VAANN")
    print("   - V = Verde (letra correcta en posición correcta)")
    print("   - A = Amarillo (letra correcta en posición incorrecta)")
    print("   - N = Negro (letra no está en la palabra)")
    print("   - Escribe 'reset' para empezar una nueva partida")
    print("   - Escribe 'salir' para terminar")
    print("="*70)
    
    while True:
        entrada = input("\nIntroduce tu intento (o 'reset'/'salir'): ").strip()
        
        if not entrada:
            continue
        
        if entrada.lower() == 'salir':
            print("\n¡Hasta luego! Gracias por usar el asistente.")
            break
        
        if entrada.lower() == 'reset':
            asistente.reset()
            print("\nAsistente reiniciado. Nueva partida iniciada.")
            continue
        
        resultado = procesar_intento(asistente, entrada)
        print(resultado)

## Ejecución principal

Contiene la función main() que orquesta todo el flujo del programa: descarga de palabras, entrenamiento del modelo, inicialización del asistente y gestión del modo interactivo.

In [ ]:
def main():
    
    # 1. Descargar y preparar palabras
    print("\n1. Descargando palabras españolas...")
    palabras_raw = descargar_palabras_api()
    palabras = filtrar_palabras_wordle(palabras_raw)
    print(f"   Total palabras válidas: {len(palabras)}")
    
    # 2. Crear estadísticas
    print("\n2. Analizando frecuencias de letras...")
    estadisticas, frecuencia_global = crear_estadisticas_letras(palabras)
    print(f"   Letras más comunes: {frecuencia_global.most_common(5)}")
    
    # 3. Preparar datos de entrenamiento
    print("\n3. Generando datos de entrenamiento...")
    X, y = generar_datos_entrenamiento(palabras, estadisticas, frecuencia_global)
    print(f"   Shape de features: {X.shape}")
    
    # 4. Entrenar modelo
    print("\n4. Entrenando modelo de red neuronal...")
    modelo = crear_modelo()
    
    # Split train/validation
    split_idx = int(0.8 * len(X))
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y[:split_idx], y[split_idx:]
    
    history = modelo.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=20,
        batch_size=32,
        verbose=1
    )
    
    # 5. Crear asistente
    print("\n5. Inicializando asistente...")
    asistente = WordleAssistant(palabras, modelo, estadisticas, frecuencia_global)
    
    print("\n" + "="*70)
    print("ASISTENTE LISTO")
    print("="*70)
    
    # Guardar modelo y datos
    print("\n6. Guardando modelo y datos...")
    modelo.save('wordle_assistant_model.keras')
    print("   Modelo guardado como 'wordle_assistant_model.keras'")
    
    with open('palabras_wordle.txt', 'w', encoding='utf-8') as f:
        f.write('\n'.join(palabras))
    print("   Palabras guardadas en 'palabras_wordle.txt'")
    
    return asistente, modelo, palabras, estadisticas, frecuencia_global, history

# Ejecutar
if __name__ == "__main__":
    asistente, modelo, palabras, estadisticas, frecuencia_global, history = main()
    
    # Ejemplo rápido
    # print("\n" + "="*70)
    # print("📖 EJEMPLO DE USO")
    # print("="*70)
    # print("\nPrueba con: CANTO/VAANN")
    # print(procesar_intento(asistente, "CANTO/VAANN"))
    
    # Reiniciar para el modo interactivo
    asistente.reset()
    
    # Iniciar modo interactivo
    print("\n" + "="*70)
    print("¿Listo para jugar?")
    iniciar = input("Presiona Enter para iniciar el modo interactivo (o escribe 'no' para salir): ")
    
    if iniciar.lower() != 'no':
        modo_interactivo(asistente)
    else:
        print("\nTIP: Puedes usar el asistente manualmente con:")
        print("  procesar_intento(asistente, 'PALABRA/COLORES')")
        print("  asistente.reset()  # Para nueva partida")


1. Descargando palabras españolas...
Descargadas 636598 palabras de API
Palabras válidas para Wordle: 10836
   Total palabras válidas: 10836

2. Analizando frecuencias de letras...
   Letras más comunes: [('A', 8814), ('E', 5378), ('O', 5337), ('S', 3844), ('R', 3666)]

3. Generando datos de entrenamiento...
   Shape de features: (10836, 33)

4. Entrenando modelo de red neuronal...
Epoch 1/20


2025-12-23 11:46:30.341107: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


271/271 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 2.4872e-04 - loss: 1.1632 - val_accuracy: 4.6125e-04 - val_loss: 0.6862
Epoch 2/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 7.7793e-04 - loss: 0.6892 - val_accuracy: 4.6125e-04 - val_loss: 0.6828
Epoch 3/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0015 - loss: 0.6791 - val_accuracy: 4.6125e-04 - val_loss: 0.6827
Epoch 4/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 7.9281e-04 - loss: 0.6788 - val_accuracy: 4.6125e-04 - val_loss: 0.6822
Epoch 5/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 5.5239e-04 - loss: 0.6750 - val_accuracy: 4.6125e-04 - val_loss: 0.6814
Epoch 6/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 1.8699e-04 - loss: 0.6753 - val_accuracy: 4.6125e-04 - val_loss: 0.6803
Epoch 7/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 6.1575e-04 - loss: 0.6755 - val_accuracy: 9.2251e-04 - val_loss: 0.6814
Epoch 8/20
271/271 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy:

Presiona Enter para iniciar el modo interactivo (o escribe 'no' para salir):  



MODO INTERACTIVO - ASISTENTE 'LA PALABRA DEL DÍA'

INSTRUCCIONES:
   - Introduce tu palabra y el resultado de colores
   - Formato: PALABRA/COLORES
   - Ejemplo: CANTO/VAANN
   - V = Verde (letra correcta en posición correcta)
   - A = Amarillo (letra correcta en posición incorrecta)
   - N = Negro (letra no está en la palabra)
   - Escribe 'reset' para empezar una nueva partida
   - Escribe 'salir' para terminar



Introduce tu intento (o 'reset'/'salir'):  SURCO/ANAAV



Intento procesado: SURCO → ANAAV

Candidatas restantes: 2

TOP 5 PALABRAS MÁS PROBABLES:
   1. CRASO (confianza: 0.6450)
   2. CRESO (confianza: 0.5579)

TOP 5 PALABRAS QUE APORTAN MÁS INFORMACIÓN:
   1. CRASO (info score: 8814.0)
   2. CRESO (info score: 5378.0)

ESTADO ACTUAL:
   Verdes: {4: 'O'}
   Amarillas: {'R', 'S', 'C'}
   Negras: {'U'}

